In [1]:
import os
import fitz
import re
import json

from ollama import Client
import faiss
import pandas as pd
import numpy as np
import Stemmer
from tqdm import tqdm
import gradio as gr

from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.retrievers import BaseRetriever, QueryFusionRetriever
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.retrievers.bm25 import BM25Retriever

# Connect to Ollama Server

In [2]:
client = Client(
  host='http://localhost:11434',
)

# Ingestion

In [7]:
# Path to the dataset folder
DATASET_PATH = '/home/jeryl4913/lto_rag_reviewer/notebooks/extracted_text.json'

def get_text_and_metadata(input_path):
    """Load text and metadata from a file and perform chunking."""
    with open(input_path, "r", encoding="utf-8") as f:
        extracted_data = json.load(f)

    texts = []
    metadata = []

    for entry in extracted_data:
        text = entry["text"]
        source_metadata = {
            "source": entry["source"],
            "folder": entry["folder"],
            "file_name": entry["file_name"],
            "page": entry["page"],
            "title": entry["title"],
            "url": entry["url"]
        }
        texts.append(text)
        metadata.append(source_metadata)

    return texts, metadata

In [8]:
docs, metadatas = get_text_and_metadata(DATASET_PATH)

In [9]:
documents = [Document(text=docs[t], metadata=metadatas[t]) for t in range(len(docs))]
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

# Embedding and Retrieval

## Dense via FAISS

In [10]:
def generate_embeddings(nodes, client, model):
    # Generate embeddings for documents using Ollama
    for doc in tqdm(nodes):
        response = client.embeddings(prompt=doc.text, model=model)
        doc.embedding = response["embedding"]
    return nodes

In [11]:
class FaissIndexer:
    """
    Faiss-based indexer for efficient similarity search using inner-product (cosine) similarity.

    This class handles the creation and management of a FAISS index from node embeddings.
    
    :ivar faiss_index: The FAISS index for storing and querying embeddings.
    :vartype faiss_index: faiss.IndexFlatIP
    :ivar embedding_dim: Dimensionality of the embeddings.
    :vartype embedding_dim: int
    """

    def __init__(self):
        """
        Initialize the FaissIndexer class.

        :ivar faiss_index: The FAISS index, initialized as None.
        :ivar embedding_dim: The dimension of embeddings, initialized as None.
        """
        self.faiss_index = None
        self.embedding_dim = None

    def normalize_embeddings(self, embeddings):
        """
        Normalize embeddings to have unit L2 norm.

        :param embeddings: Array of embeddings to normalize.
        :type embeddings: np.ndarray
        :return: Normalized embeddings.
        :rtype: np.ndarray
        """
        return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    def build_index(self, nodes):
        """
        Build the FAISS index from a list of nodes containing embeddings.

        :param nodes: List of nodes, where each node contains an `embedding` attribute.
        :type nodes: list
        :raises ValueError: If the nodes list is empty or embeddings are inconsistent.
        """
        if not nodes:
            raise ValueError("Nodes list cannot be empty.")
        
        embeddings = np.array([np.array(node.embedding) for node in nodes])
        normalized_embeddings = self.normalize_embeddings(embeddings)

        self.embedding_dim = normalized_embeddings[0].shape[0]
        self.faiss_index = faiss.IndexFlatIP(self.embedding_dim)  # Inner-product similarity
        self.faiss_index.add(normalized_embeddings)

    def get_index(self):
        """
        Get the FAISS index instance.

        :return: The FAISS index used for similarity search.
        :rtype: faiss.IndexFlatIP
        :raises ValueError: If the index has not been built.
        """
        if self.faiss_index is None:
            raise ValueError("Index has not been built yet. Call 'build_index' first.")
        return self.faiss_index

In [12]:
class FAISSVectorStoreRetriever(BaseRetriever):
    def __init__(self, faiss_index, documents):
        """
        Initialize the FAISS retriever.
        :param faiss_index: The FAISS index containing precomputed embeddings.
        :param documents: List of document chunks.
        :param embeddings: Precomputed embeddings corresponding to the document chunks.
        """
        self.faiss_index = faiss_index
        self.documents = documents

    def _retrieve(self, query_embedding, top_k=5):
        """
        Retrieve the top-k nearest neighbors using the FAISS index.
        :param query_embedding: The embedding of the query.
        :param top_k: Number of top results to retrieve.
        """

        norm_query_embedding = np.array([query_embedding])
        norm_query_embedding /= np.linalg.norm(norm_query_embedding, axis=1, keepdims=True)

        distances, indices = self.faiss_index.search(norm_query_embedding, top_k)
        retrieved_docs = [
            NodeWithScore(node=self.documents[idx], score=1 - dist)
            for idx, dist in zip(indices[0], distances[0])
            if idx != -1
        ]
        return retrieved_docs

In [13]:
# embedding
nodes_embed = generate_embeddings(nodes, client, "mxbai-embed-large")

100%|██████████| 12302/12302 [11:03<00:00, 18.54it/s]


In [14]:
#indexing
index = FaissIndexer()
index.build_index(nodes_embed)
faiss_index = index.get_index()

faiss_retriever = FAISSVectorStoreRetriever(faiss_index=faiss_index,documents=nodes_embed)

## Sparse Embedding via BM25

In [15]:
# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=nodes,
#     similarity_top_k=5,
#     stemmer=Stemmer.Stemmer("english"),
#     language="english",
# )

## Hybrid Retrieval via Reciprocal Rank

In [16]:
def hybrid_embedding(results: dict, top_k: int):
    x = QueryFusionRetriever
    ranked_results = QueryFusionRetriever._reciprocal_rerank_fusion(x, results)
    return ranked_results[:top_k]

# Post Retrieval

## Summarization

In [17]:
def summarize_each_chunk(nodes, client, query, model="llama3.3", parent=False):
    if parent:
        chunks = [doc.text for doc in nodes]
    else:
        chunks = [doc.node.text for doc in nodes]
    summaries = []
    
    for i, chunk in enumerate(chunks):
        prompt = f"""
        Summarize the following text in one concise paragraph, focusing on key points relevant to the query: "{query}".
        
        - Emphasize information directly related to the query.
        - Exclude unrelated, redundant, or speculative details.
        - Do NOT introduce new information or answer the query itself. 
        
        Text:
        {chunk}
        
        Summary:
        """
        
        response = client.generate(model=model, prompt=prompt)
        summary = response['response'].strip()
        summaries.append(summary)

    return summaries

# Generation

In [18]:


def generate_response_with_notice(summaries, query, choices, client, model="llama3.3"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."

    Summarized Context:
    {context}
    
    Query:
    {query}
    {choices}
    
    \nPlease answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

# Querying

## Query Transforms

In [19]:
import re
import json

prompt = f"""
    You are an expert assistant. Below are examples of how to decompose a query
    into three sub-queries, and also provide short answers for each sub-query.
    Finally, place these into a JSON array.

    Example #1:
    USER: "Why is the sky blue?"
    SUB-QUERY 1: "Explain Rayleigh scattering"
    ANSWER 1: "Rayleigh scattering is the scattering of light by particles in the atmosphere..."
    SUB-QUERY 2: "How light interacts with molecules?"
    ANSWER 2: "Light interacts with molecules through absorption and scattering, altering wavelengths..."
    SUB-QUERY 3: "How human eyes perceive color?"
    ANSWER 3: "The human eye detects the shorter, scattered wavelengths as blue..."

    FINAL ANSWER (JSON array):
    [
    {{
        "sub_query": "Explain Rayleigh scattering",
        "answer": "Rayleigh scattering is the scattering of light by particles in the atmosphere..."
    }},
    {{
        "sub_query": "How light interacts with molecules?",
        "answer": "Light interacts with molecules through absorption and scattering, altering wavelengths..."
    }},
    {{
        "sub_query": "How human eyes perceive color?",
        "answer": "The human eye detects the shorter, scattered wavelengths as blue..."
    }}
    ]

    Now your turn:
    USER: "{"What are the requirements in getting a license?"}"

    FINAL ANSWER (JSON array):
    """

response = client.generate(model="llama3.3", prompt=prompt)
print(response.response)

# Extract the JSON part using regex
match = re.search(r'\[\s*{.*?}\s*\]', response.response, re.S)
if match:
    json_text = match.group(0)
    parsed_json = json.loads(json_text)
    print(json.dumps(parsed_json, indent=4))
else:
    print("No JSON found.")

To decompose the query "What are the requirements in getting a license?" into three sub-queries and provide short answers for each, we can break it down as follows:

1. **SUB-QUERY 1**: "What are the general eligibility criteria?"
   - **ANSWER 1**: "General eligibility criteria include age requirements, residency, and background checks, which vary by jurisdiction and type of license."

2. **SUB-QUERY 2**: "What documents are typically required for a license application?"
   - **ANSWER 2**: "Typically required documents include proof of identity, proof of residency, social security number or equivalent, and sometimes proof of qualifications or certifications relevant to the license being applied for."

3. **SUB-QUERY 3**: "Are there any tests or exams that must be passed to obtain a license?"
   - **ANSWER 3**: "Yes, many licenses require passing a written, practical, or both types of exams to demonstrate competence in the area the license is for, such as driving tests for a driver's l

## Query Generation

In [20]:
docstore = {}

# Store documents using full metadata as the key
for doc in documents:
    key = tuple(doc.metadata.items())  # Convert metadata to tuple for hashable key
    docstore[key] = doc

In [21]:
def get_document_by_chunk_metadata(chunk_node):
    # Convert chunk metadata to tuple for matching
    metadata_key = tuple(chunk_node.metadata.items())

    # Retrieve document from docstore
    document = docstore.get(metadata_key)
    return document

In [22]:
def remove_duplicate_documents(doc_list):
    seen_ids = set()
    unique_docs = []

    for doc in doc_list:
        if doc.doc_id not in seen_ids:
            seen_ids.add(doc.doc_id)
            unique_docs.append(doc)

    return unique_docs

In [23]:
import json

def gen_query(query, choices, top_k, client, mode='dense', summary=False, model="llama3.3", chunks_only=False):
    prompt = f"""
    You are an expert assistant. Below are examples of how to decompose a query
    into three sub-queries, and also provide short answers for each sub-query.
    Finally, place these into a JSON array.

    Example #1:
    USER: "Why is the sky blue?"
    SUB-QUERY 1: "Explain Rayleigh scattering"
    ANSWER 1: "Rayleigh scattering is the scattering of light by particles in the atmosphere..."
    SUB-QUERY 2: "How light interacts with molecules?"
    ANSWER 2: "Light interacts with molecules through absorption and scattering, altering wavelengths..."
    SUB-QUERY 3: "How human eyes perceive color?"
    ANSWER 3: "The human eye detects the shorter, scattered wavelengths as blue..."

    FINAL ANSWER (JSON array):
    [
    {{
        "sub_query": "Explain Rayleigh scattering",
        "answer": "Rayleigh scattering is the scattering of light by particles in the atmosphere..."
    }},
    {{
        "sub_query": "How light interacts with molecules?",
        "answer": "Light interacts with molecules through absorption and scattering, altering wavelengths..."
    }},
    {{
        "sub_query": "How human eyes perceive color?",
        "answer": "The human eye detects the shorter, scattered wavelengths as blue..."
    }}
    ]

    Now your turn:
    USER: "{query}"

    FINAL ANSWER (JSON array):
    """

    response_text = client.generate(model=model, prompt=prompt)

    # Extract the JSON part using regex
    match = re.search(r'\[\s*{.*?}\s*\]', response_text.response, re.S)
    if match:
        json_text = match.group(0)
        parsed_json = json.loads(json_text)
        print(json.dumps(parsed_json, indent=4))
        json_text = json.dumps(parsed_json)
        embed_result = client.embeddings(prompt=json_text, model="mxbai-embed-large")
    else:
        print("No JSON found.")
        embed_result = client.embeddings(prompt=query, model="mxbai-embed-large")


    # Return both the parsed sub-queries/answers and the embedding vector
    query_embedding = embed_result["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_notice(context, query, choices, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references), context

# Evaluation

In [24]:
# Generate prompts dynamically
def generate_choices(row):
    options = []
    for choice in ['A', 'B', 'C', 'D', 'E']:
        # Check for NaN or blank values
        if pd.notna(row[choice]) and row[choice] != '':
            options.append(f"{choice}. {row[choice]}")
    
    # Construct the prompt with few-shot examples
    choices = "\n".join(options)
    
    return choices

In [25]:
from sklearn.model_selection import train_test_split
# Load the Excel file
file_path = '/home/jeryl4913/lto_rag_reviewer/notebooks/eval/LTO_EXAM.csv'
df = pd.read_csv(file_path)

# Display the first few rows of each set
print("Testing Data:")
print(len(df))
display(df.head())

Testing Data:
60


,Question,A,B,C,D,E,Answer,Choices
0,"If the driver is turning left, he must: (U-tur...",have the right of way,do so slowly with caution,yield to approaching cars,NaN,NaN,C,A. have the right of way\nB. do so slowly with...
1,"When a vehicle starts to skid, what should the...",Immediately step on the brakes,Hold firmly on to the wheel while slowing down...,Turn the wheels tp the opposite the direction ...,NaN,NaN,B,A. Immediately step on the brakes\nB. Hold fir...
2,"If you are parking uphill without a curb, turn...",edge of the street,other side of the street,middle of the street,NaN,NaN,A,A. edge of the street\nB. other side of the st...
3,"When parking downhill, you should turn your fr...",toward the curb of the sidewalk,away from the curb,any direction will do,NaN,NaN,A,A. toward the curb of the sidewalk\nB. away fr...
4,Which of the following is the maximum speed li...,60 kph,80 kph,100 kph,NaN,NaN,C,A. 60 kph\nB. 80 kph\nC. 100 kph


In [26]:
import time

df["AI"] = np.nan
ai_answer = []
context_answer = []
inference_times = []
for i in tqdm(range(len(df))):
    start_time = time.time()
    answ = gen_query(df["Question"].iloc[i], df["Choices"].iloc[i], top_k=15, client=client, mode='dense', model="llama3.3")
    ai_answer.append(answ[0])
    context_answer.append(answ[2])
    end_time = time.time()
    inference_times.append(end_time - start_time)

df["AI"] = ai_answer
df["Inference_times"] = inference_times
df["Context"] = context_answer
print("Updated Testing Data with AI Answers:")
display(df.head())

  0%|          | 0/60 [00:00<?, ?it/s]

[
    {
        "sub_query": "What are the basic traffic rules for turning?",
        "answer": "Basic traffic rules for turning include yielding to pedestrians and oncoming traffic, using turn signals, and following traffic signals."
    },
    {
        "sub_query": "How do left turn rules specifically apply in this scenario?",
        "answer": "When turning left, drivers must yield to oncoming traffic and pedestrians, signal their intention to turn, and ensure it's safe to proceed."
    },
    {
        "sub_query": "What is the difference between a U-turn and a left turn in terms of driving rules?",
        "answer": "A U-turn involves turning around to head in the opposite direction, often from a divided highway or where permitted by signs. In contrast, a left turn involves changing direction at an intersection or into another road without reversing direction."
    }
]
using FAISS


  2%|▏         | 1/60 [01:19<1:17:44, 79.06s/it]

[
    {
        "sub_query": "Explain what causes a vehicle to skid",
        "answer": "A vehicle skids when its tires lose traction with the road surface, often due to speeding, sudden turns, or driving on slippery roads."
    },
    {
        "sub_query": "Describe corrective steering techniques during a skid",
        "answer": "To correct a skid, take your foot off the accelerator, avoid slamming on the brakes, and steer gently in the direction you want the vehicle to go."
    },
    {
        "sub_query": "How to recover from a skid safely",
        "answer": "Recover from a skid by easing off corrective steering once the vehicle straightens out, then gradually resume speed when it's safe to do so, avoiding overcorrection."
    }
]
using FAISS


  3%|▎         | 2/60 [02:09<1:00:15, 62.33s/it]

...

 98%|█████████▊| 59/60 [56:22<00:54, 54.91s/it]

[
    {
        "sub_query": "What are common reasons for no-parking zones?",
        "answer": "No-parking zones are designated due to safety concerns, traffic flow, emergency access needs, and to maintain clear paths for pedestrians and vehicles."
    },
    {
        "sub_query": "How do parking restrictions vary by location?",
        "answer": "Parking restrictions can significantly vary by location, including differences in urban vs. rural areas, near schools, hospitals, or in areas with high foot traffic, each with its unique set of rules and regulations."
    },
    {
        "sub_query": "What are the consequences of parking in a no-parking zone?",
        "answer": "Consequences can include fines, towing of the vehicle, and in some cases, points added to one's driving license, depending on the jurisdiction and severity of the infraction."
    }
]
using FAISS


100%|██████████| 60/60 [57:21<00:00, 57.36s/it]

Updated Testing Data with AI Answers:


,Question,A,B,C,D,E,Answer,Choices,AI,Inference_times,Context
0,"If the driver is turning left, he must: (U-tur...",have the right of way,do so slowly with caution,yield to approaching cars,NaN,NaN,C,A. have the right of way\nB. do so slowly with...,[C],79.057585,"{vehicle being operated on a ""through highway""..."
1,"When a vehicle starts to skid, what should the...",Immediately step on the brakes,Hold firmly on to the wheel while slowing down...,Turn the wheels tp the opposite the direction ...,NaN,NaN,B,A. Immediately step on the brakes\nB. Hold fir...,[B],50.625948,{143 Dealing with Emergency Situations SKIDDIN...
2,"If you are parking uphill without a curb, turn...",edge of the street,other side of the street,middle of the street,NaN,NaN,A,A. edge of the street\nB. other side of the st...,[A],53.014183,{143 Dealing with Emergency Situations SKIDDIN...
3,"When parking downhill, you should turn your fr...",toward the curb of the sidewalk,away from the curb,any direction will do,NaN,NaN,A,A. toward the curb of the sidewalk\nB. away fr...,[A],69.176013,{143 Dealing with Emergency Situations SKIDDIN...
4,Which of the following is the maximum speed li...,60 kph,80 kph,100 kph,NaN,NaN,C,A. 60 kph\nB. 80 kph\nC. 100 kph,[C],52.722611,"{potential for increased traffic density, driv..."


In [27]:
import re


def process_answers(answers):
    formatted_answers = []
    
    for a in answers:
        
        matches = re.findall(r'\[?\s*([A-E](?:\s*,\s*[A-E])*)\s*\]?', str(a)) # Extract answers like [A, C, D] or [A] or [B, D]
        answers = []
        for match in matches:
            answers.extend(re.split(r'\s*,\s*', match))  # Split by comma and remove spaces
        unique_sorted_answers = sorted(set(answers), key=lambda x: ['A', 'B', 'C', 'D', 'E'].index(x))
        if not unique_sorted_answers:
            formatted_answers.append(None)
        else:
            formatted_answers.append(unique_sorted_answers)
    return formatted_answers

df_results = df[["Question", "Answer", "AI", "Inference_times"]].copy()
df_results['Answer'] = df_results['Answer'].apply(lambda x: x.split(', '))
df_results['AI'] = process_answers(df_results["AI"])
df_results['Answer'] = process_answers(df_results["Answer"])



def calculate_scores(df):
    scores = []
    for index, row in df.iterrows():
        correct_answers = set(row['Answer'] if row['Answer'] is not None else [])
        ai_answers = set(row['AI'] if row['AI'] is not None else [])
        if ai_answers == correct_answers:
            score = 1.0
        else:
            score = 0.0
        scores.append(score)
    
    df['Score'] = scores
    accuracy = scores.count(1.0) / len(scores)
    print(f'Final Score: {scores.count(1.0):.2f}/{len(scores):.2f}')
    print(f'Accuracy: {accuracy:.2f}%')
    return df

# Apply the scoring function
scored_df = calculate_scores(df_results)

# Display the dataframe to verify the results
display(scored_df[['Question', 'Answer', 'AI', 'Score', 'Inference_times']])

Final Score: 48.00/60.00
Accuracy: 0.80%


,Question,Answer,AI,Score,Inference_times
0,"If the driver is turning left, he must: (U-tur...",[C],[C],1.0,79.057585
1,"When a vehicle starts to skid, what should the...",[B],[B],1.0,50.625948
2,"If you are parking uphill without a curb, turn...",[A],[A],1.0,53.014183
3,"When parking downhill, you should turn your fr...",[A],[A],1.0,69.176013
4,Which of the following is the maximum speed li...,[C],[C],1.0,52.722611
5,Never park or stop at the side of the road wit...,[C],[B],0.0,65.410310
6,While driving with maximum speed and you have ...,[C],[C],1.0,62.963746
7,What will happen when your rear tire blows out?,[B],[A],0.0,39.967454
8,Operating a public utility vehicle equipped wi...,[A],[B],0.0,54.254286
9,Your speed while driving at night should keep on:,[A],[A],1.0,53.877651


In [28]:
# Generate prompts dynamically
def generate_prompt(row):
    
    # Construct the prompt with few-shot examples
    prompt = f"\nActual Question: {row['Question']}\n" 
    prompt += "\nPlease answer the question based on the given context."
    
    return prompt

In [29]:
# Load the Excel file
file_path = '/home/jeryl4913/lto_rag_reviewer/notebooks/eval/LTO_EXAM_QnA.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')
df['Prompt'] = df.apply(generate_prompt, axis=1)
display(df.head())

,Question,Answer,Prompt
0,Traffic Jam can be prevented if you,Keep opposing lanes open,\nActual Question: Traffic Jam can be prevente...
1,When making a right turn you should,Stay on the outermost lane of the road then si...,\nActual Question: When making a right turn yo...
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,\nActual Question: When you intend to turn rig...
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,\nActual Question: At an intersection with a t...
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,\nActual Question: Graft and corruption in the...


In [30]:
def generate_response_with_notice(summaries, query, client, model="llama3.3"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."

    Summarized Context:
    {context}
    
    Query:
    {query}
    
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

In [33]:
import json

def gen_query(query, top_k, client, mode='dense', summary=False, model="llama3.3", chunks_only=False):
    prompt = f"""
    You are an expert assistant. Below are examples of how to decompose a query
    into three sub-queries, and also provide short answers for each sub-query.
    Finally, place these into a JSON array.

    Example #1:
    USER: "Why is the sky blue?"
    SUB-QUERY 1: "Explain Rayleigh scattering"
    ANSWER 1: "Rayleigh scattering is the scattering of light by particles in the atmosphere..."
    SUB-QUERY 2: "How light interacts with molecules?"
    ANSWER 2: "Light interacts with molecules through absorption and scattering, altering wavelengths..."
    SUB-QUERY 3: "How human eyes perceive color?"
    ANSWER 3: "The human eye detects the shorter, scattered wavelengths as blue..."

    FINAL ANSWER (JSON array):
    [
    {{
        "sub_query": "Explain Rayleigh scattering",
        "answer": "Rayleigh scattering is the scattering of light by particles in the atmosphere..."
    }},
    {{
        "sub_query": "How light interacts with molecules?",
        "answer": "Light interacts with molecules through absorption and scattering, altering wavelengths..."
    }},
    {{
        "sub_query": "How human eyes perceive color?",
        "answer": "The human eye detects the shorter, scattered wavelengths as blue..."
    }}
    ]

    Now your turn:
    USER: "{query}"

    FINAL ANSWER (JSON array):
    """

    response_text = client.generate(model=model, prompt=prompt)

    # Extract the JSON part using regex
    match = re.search(r'\[\s*{.*?}\s*\]', response_text.response, re.S)
    if match:
        json_text = match.group(0)
        parsed_json = json.loads(json_text)
        print(json.dumps(parsed_json, indent=4))
        json_text = json.dumps(parsed_json)
        embed_result = client.embeddings(prompt=json_text, model="mxbai-embed-large")
    else:
        print("No JSON found.")
        embed_result = client.embeddings(prompt=query, model="mxbai-embed-large")


    # Return both the parsed sub-queries/answers and the embedding vector
    query_embedding = embed_result["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_notice(context, query, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references), context

In [34]:
import time
qr_range = (0,60)
df["AI"] = np.nan
ai_answer = [] 
inference_times = []

for i in tqdm(range(*qr_range)):
    start_time = time.time()
    ai_answer.append(gen_query(df.loc[i,"Prompt"],  top_k=15, client=client, mode='dense', model="llama3.3"))
    end_time = time.time()
    inference_times.append(end_time - start_time)


  0%|          | 0/60 [00:00<?, ?it/s]

[
    {
        "sub_query": "What are the primary causes of traffic jams?",
        "answer": "Traffic jams are primarily caused by factors such as rush hour congestion, road accidents, construction, and special events that increase traffic volume beyond what the roads can handle."
    },
    {
        "sub_query": "How does infrastructure planning affect traffic flow?",
        "answer": "Infrastructure planning significantly affects traffic flow. Well-designed roads with adequate lanes, smart intersections, and efficient public transportation systems can reduce congestion by managing traffic volume more effectively."
    },
    {
        "sub_query": "What role do driver behavior and technology play in preventing traffic jams?",
        "answer": "Driver behavior, such as following speed limits and avoiding sudden stops, and technological innovations like real-time traffic monitoring and smart traffic lights can significantly reduce congestion by optimizing traffic flow and minimizi

  2%|▏         | 1/60 [01:15<1:14:28, 75.74s/it]

[
    {
        "sub_query": "What is the standard procedure for signaling turns while driving?",
        "answer": "The standard procedure involves using turn signals (blinkers) to indicate your intention to turn, which helps other drivers anticipate and react to your actions."
    },
    {
        "sub_query": "How should you position your vehicle when preparing to make a right turn?",
        "answer": "When making a right turn, you should move into the right lane or the right-most portion of your lane, signaling your intention and checking mirrors and blind spots for other vehicles or pedestrians."
    },
    {
        "sub_query": "What are key safety considerations when executing a right turn?",
        "answer": "Key safety considerations include yielding to pedestrians, bicyclists, or other vehicles that may have the right of way, maintaining a safe speed, and being aware of potential hazards such as intersections, traffic signals, or road signs."
    }
]
using FAISS


  3%|▎         | 2/60 [02:30<1:12:36, 75.12s/it]

...

 97%|█████████▋| 58/60 [1:05:29<01:53, 56.73s/it]

[
    {
        "sub_query": "What is the purpose of a railway crossing mark?",
        "answer": "The mark for a railway crossing is crucial for alerting drivers to the presence of a railroad crossing, enhancing safety by prompting caution."
    },
    {
        "sub_query": "How does the design of railway crossing marks contribute to safety?",
        "answer": "The design typically includes stripes or crossbucks with specific colors and patterns that are internationally recognized, aiming to grab the attention of road users and clearly indicate where to stop or proceed with caution."
    },
    {
        "sub_query": "What legal implications or standards govern the use of railway crossing marks?",
        "answer": "Standards for railway crossing marks vary by country but often include regulations on placement, maintenance, and visibility, with many countries adhering to guidelines set by international bodies to ensure consistency and safety."
    }
]
using FAISS


 98%|█████████▊| 59/60 [1:06:21<00:55, 55.55s/it]

[
    {
        "sub_query": "What are the legal requirements for using child restraints in vehicles?",
        "answer": "The legal requirements typically mandate that children under a certain age or height must be secured in an approved child restraint system when traveling in a vehicle."
    },
    {
        "sub_query": "How do child restraints improve safety for children during travel?",
        "answer": "Child restraints significantly reduce the risk of fatal injury by properly securing the child and distributing the force of sudden stops or crashes across the stronger parts of the child's body."
    },
    {
        "sub_query": "What are the guidelines for correctly installing a child restraint in a vehicle?",
        "answer": "Correct installation involves following the manufacturer's instructions, ensuring the seat is securely fastened to the vehicle, and adjusting the harness to fit the child snugly without excessive slack."
    }
]
using FAISS


100%|██████████| 60/60 [1:07:07<00:00, 67.13s/it]


In [35]:
df.loc[qr_range[0]:qr_range[1]-1, "AI"] = [answ[0] for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Context"] = [" ".join(map(str, answ[2])) for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Inference_Time"] = inference_times 
df_new = df.loc[qr_range[0]:qr_range[1]-1].copy()
df = df_new.copy()

/tmp/ipykernel_4770/3496959586.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['To prevent traffic jams, consider the following:\n\n1. **Plan Ahead**: Ensure you know your route and give yourself plenty of time to reach your destination to avoid feeling rushed.\n2. **Use Alternative Routes**: Familiarize yourself with alternative routes to your destination in case of heavy traffic.\n3. **Carpool**: Reduce vehicle volume on the road by carpooling whenever possible.\n4. **Avoid Peak Hours**: If possible, adjust your departure time to avoid peak hours when traffic is usually heavier.\n5. **Stay Calm and Patient**: Maintain a calm demeanor while driving, as stress and frustration can lead to unsafe driving behaviors that might exacerbate traffic congestion.\n\nBy adopting these strategies, you can contribute to reducing the likelihood of traffic jams and make your driving experience safer and less stres

# 10. Similarity Evaluation

In [36]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator
from llama_index.core.base.embeddings.base import BaseEmbedding
import asyncio
from llama_index.core.embeddings import resolve_embed_model
from pydantic import PrivateAttr

class OllamaEmbeddingModel(BaseEmbedding):
    _client: Client = PrivateAttr()

    def __init__(self, model_name: str = "mxbai-embed-large", timeout: int = 300):
        super().__init__()
        self.model_name = model_name
        self._client = Client() 

    async def _aget_query_embedding(self, query: str) -> list[float]:
        return await self._aget_text_embedding(query)

    async def _aget_text_embedding(self, text: str) -> list[float]:
        loop = asyncio.get_event_loop()
        embedding_response = await loop.run_in_executor(
            None, self._client.embeddings, self.model_name, text
        )
        return embedding_response['embedding']  

    def _get_query_embedding(self, query: str) -> list[float]:
        return self._get_text_embedding(query)

    def _get_text_embedding(self, text: str) -> list[float]:
        embedding_response = self._client.embeddings(
            model=self.model_name,
            prompt=text
        )
        return embedding_response['embedding']


embed_model = OllamaEmbeddingModel(model_name="mxbai-embed-large")
evaluator = SemanticSimilarityEvaluator(
    embed_model=embed_model,
    similarity_threshold=0.6
)

results_scores = []
results_passing = []
for i in tqdm(range(len(df))):
    response = df.loc[i, "AI"]
    reference = df.loc[i, "Answer"]

    result = await evaluator.aevaluate(
    response=response,
    reference=reference,
    )
    results_scores.append(result.score)
    results_passing.append(result.passing)
    
df['Score'] = results_scores
df['Passing'] = results_passing

average_score = df['Score'].mean()
total_items = len(df)
passing_items = df['Passing'].sum()  
print(f"Average Score: {average_score:.4f}")
print(f"Passing: {passing_items}/{total_items}") 
display(df[['Question', 'Answer', 'AI', 'Score', 'Passing', 'Inference_Time']])

100%|██████████| 60/60 [00:04<00:00, 12.93it/s]

Average Score: 0.6664
Passing: 47/60


,Question,Answer,AI,Score,Passing,Inference_Time
0,Traffic Jam can be prevented if you,Keep opposing lanes open,"To prevent traffic jams, consider the followin...",0.659305,True,75.737471
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"When making a right turn, you should signal ea...",0.744879,True,74.681246
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,30 meters before the turn.,0.862716,True,73.718779
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"At an intersection with a traffic light, make ...",0.780225,True,63.052880
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,To eliminate graft and corruption in the traff...,0.631648,True,85.213734
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,"On a four-lane road with a single white line, ...",0.806113,True,74.987857
6,A double solid yellow line with broken white l...,Absolutely no overtaking,A double solid yellow line typically signifies...,0.494653,False,108.738519
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,"When making a U-turn, you should start turning...",0.730976,True,64.388900
8,Signs that are triangular in shape and with a ...,Caution or warning signs,"Based on the context provided, signs that are ...",0.687599,True,63.012042
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,Warning signs or Hazard markers are typically ...,0.748237,True,88.442191


# 11. Relevancy Evaluation

In [41]:
from llama_index.core.evaluation import RelevancyEvaluator
from llama_index.llms.ollama import Ollama

ollama_llm = Ollama(model="llama3.1:8b", request_timeout=300)
evaluator = RelevancyEvaluator(llm=ollama_llm)

eval_results = []

for i in tqdm(range(len(df))):
    eval_result = await evaluator.aevaluate(
        query=df.loc[i, "Question"],
        response=df.loc[i, "AI"],
        contexts=[df.loc[i, "Context"]]  
    )
    eval_results.append(eval_result.passing)

df['Eval'] = eval_results

total_items = len(df)
passing_items = df['Eval'].sum()
score = f"Score: {passing_items}/{total_items}"
percentage = passing_items / total_items if total_items > 0 else 0
print(score)
print(f"Percentage: {percentage:.2%}")
display(df[['Question', 'Answer', 'AI', 'Eval', 'Inference_Time']])

  0%|          | 0/60 [00:00<?, ?it/s]

100%|██████████| 60/60 [05:01<00:00,  5.02s/it]

Score: 58/60
Percentage: 96.67%


,Question,Answer,AI,Eval,Inference_Time
0,Traffic Jam can be prevented if you,Keep opposing lanes open,"To prevent traffic jams, consider the followin...",True,75.737471
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"When making a right turn, you should signal ea...",True,74.681246
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,30 meters before the turn.,True,73.718779
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"At an intersection with a traffic light, make ...",True,63.052880
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,To eliminate graft and corruption in the traff...,True,85.213734
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,"On a four-lane road with a single white line, ...",True,74.987857
6,A double solid yellow line with broken white l...,Absolutely no overtaking,A double solid yellow line typically signifies...,False,108.738519
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,"When making a U-turn, you should start turning...",True,64.388900
8,Signs that are triangular in shape and with a ...,Caution or warning signs,"Based on the context provided, signs that are ...",True,63.012042
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,Warning signs or Hazard markers are typically ...,True,88.442191
